# Jobs Data Integration

Libraries used:
* pandas
* numpy

## Introduction
Merge the cleaned dataset with new dataset that are from 'www.jobhuntlisting.com'. However, the dataset are in different format, **schematic mapping** is applied to resolve the conflict before merge. Data conflicts are also be considered and handle.


##  Import libraries 

In [4]:
import pandas as pd
import numpy as np

# display configuration
from IPython.display import display

pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  

### 1. Examining and loading data
Examine "dataset2.csv". 

In [6]:
df = pd.read_csv('dataset2.csv')

df.head(5)

,Location,Job Title,Monthly Payment,Closing,Category,Type,Opening,Organisation,Full-Time Equivalent (FTE)
0,Berkshire,Lead CRA UK,4583.33,2012-03-08 12:00:00,Health,NaN,2012-01-08 12:00:00,SEC Recruitment,1.0
1,Bristol,Possession Manager,2812.50,2013-09-06 12:00:00,Engineering,Permanent,2013-08-07 12:00:00,Navartis Limited,1.0
2,Coventry,NVQ Assessor Banking/Financial Services Salary...,1791.67,2013-05-02 00:00:00,Hospitality,Permanent,2013-02-01 00:00:00,Pertemps,1.0
3,Poole,HEAD OF CARE (RGN/RMN) Poole,2916.67,2013-09-24 15:00:00,Health,NaN,2013-07-26 15:00:00,Team 24 Ltd,1.0
4,Woking,SQL Server DBA,3125.00,2012-12-21 12:00:00,Information Technology,NaN,2012-11-21 12:00:00,Matchtech,1.0


In [7]:
print('\n____________ Dataset info ____________')
display(df.info())              
print('\n____________ Statistics of numerical features ____________')
display(df.describe())  
print('\n____________ Statistics of literal features ____________')
display(df.describe(include=[object]))


____________ Dataset info ____________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Location                    5000 non-null   object 
 1   Job Title                   5000 non-null   object 
 2   Monthly Payment             5000 non-null   float64
 3   Closing                     5000 non-null   object 
 4   Category                    5000 non-null   object 
 5   Type                        3593 non-null   object 
 6   Opening                     5000 non-null   object 
 7   Organisation                4531 non-null   object 
 8   Full-Time Equivalent (FTE)  5000 non-null   float64
dtypes: float64(2), object(7)
memory usage: 351.7+ KB


None


____________ Statistics of numerical features ____________


,Monthly Payment,Full-Time Equivalent (FTE)
count,5000.000000,5000.00000
mean,2854.147972,0.98560
std,1310.434763,0.09236
min,424.000000,0.20000
25%,1916.670000,1.00000
50%,2583.330000,1.00000
75%,3541.670000,1.00000
max,8000.000000,1.00000



____________ Statistics of literal features ____________


,Location,Job Title,Closing,Category,Type,Opening,Organisation
count,5000,5000,5000,5000,3593,5000,4531
unique,431,5000,2062,8,2,1967,2481
top,UK,Lead CRA UK,2013-01-11 12:00:00,Information Technology,Permanent,2013-12-09 00:00:00,UKStaffsearch
freq,753,1,8,1251,3034,12,52


##### Import the cleaned dataset from previous task

In [9]:
df_dataset1 = pd.read_csv('cleaned_dataset1.csv')

df_dataset1.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,68677473,Fluent Polish Speaking Incident Manager Krakow...,UK,Proxime,non-specified,permanent,IT Jobs,6000.0,2013-11-02 15:00:00,2014-01-31 15:00:00,totaljobs.com
1,71851065,GIS Trainer Central London Permanent ****k*...,London,Locum Staffing Ltd,non-specified,permanent,IT Jobs,40000.0,2012-04-30 15:00:00,2012-05-30 15:00:00,totaljobs.com
2,68665487,Trafficker/Ad Op's Manager,South East London,Savvy Media Group,non-specified,permanent,IT Jobs,40000.0,2012-08-27 15:00:00,2012-10-26 15:00:00,totaljobs.com
3,68685701,PR Associate Director,Oxfordshire,Pure Recruitment,non-specified,contract,"PR, Advertising & Marketing Jobs",55000.0,2012-03-15 15:00:00,2012-03-29 15:00:00,totaljobs.com
4,66960212,Accounts Payable Administrator,Littlehampton,The Recruitment Consultancy Ltd,non-specified,permanent,Accounting & Finance Jobs,35500.0,2012-07-23 00:00:00,2012-10-21 00:00:00,totaljobs.com


In [10]:
df_dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50753 entries, 0 to 50752
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            50753 non-null  int64  
 1   Title         50753 non-null  object 
 2   Location      50753 non-null  object 
 3   Company       48882 non-null  object 
 4   ContractType  50753 non-null  object 
 5   ContractTime  50753 non-null  object 
 6   Category      50753 non-null  object 
 7   Salary        50753 non-null  float64
 8   OpenDate      50753 non-null  object 
 9   CloseDate     50753 non-null  object 
 10  SourceName    50753 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 4.3+ MB


### 2. Resolving schema conflicts
Give some general information about the conflicts you found in two schemas and steps you taken to resolve them.

#### 2.1. Id 
Generate 8 digits Id for `df`.

In [13]:
ids = np.random.randint(10000000, 100000000, size=len(df))

In [14]:
# assign Id 
df['Id'] = ids

#### 2.2. Columns Name
- Certain columns share the same value type with `df_dataset1` schema => rename those columns
##### Rename Columns

In [16]:
# rename the column following the schema of the previous df
df.rename(columns = {'Job Title':'Title', 'Closing':'CloseDate', 'Type':'ContractTime',
                             'Closing':'CloseDate', 'Opening': 'OpenDate', 'Organisation' : 'Company'}, inplace = True)

#### 2.3. Contract Type
Use `Full-Time-Equivalent` to determine the contract type:

- **1.0** means the jobs are full-time employment.
- **others** means the jobs are part-time employment.

In [18]:
# fill the ContractType
df['ContractType'] = df['Full-Time Equivalent (FTE)'].apply(lambda x: 'full_time' if x == 1.0 else 'part_time')

In [19]:
# examine the ContractType values
df['ContractType'].value_counts()

ContractType
full_time    4857
part_time     143
Name: count, dtype: int64

#### 2.4: Contract Time
Check whether there are missing values. If there are any, fill those with **'non-specified'**

In [21]:
# examine ContractTime missing values
df['ContractTime'].isna().sum()

1407

In [22]:
# fill the missing values
df['ContractTime'].fillna('non-specified',inplace=True)

Check the values of the `ContractTime`

In [24]:
df['ContractTime'].value_counts()

ContractTime
Permanent              3034
non-specified          1407
Fixed Term Contract     559
Name: count, dtype: int64

The values are not in the schema format:

- **Permanent** need to be changed to **permanent**
- **Fixed Term Contract** need to be changed to **contract**

In [26]:
contract_time_dict = {'Permanent':'permanent','Fixed Term Contract': 'contract'}

df['ContractTime'].replace(contract_time_dict,inplace=True)

# examine the fixed ContractTime
df['ContractTime'].value_counts()

ContractTime
permanent        3034
non-specified    1407
contract          559
Name: count, dtype: int64

#### 2.5. Category

In [28]:
df['Category'].value_counts()

Category
Information Technology    1251
Health                     804
Engineering                732
Finance                    675
Sales                      517
Hospitality                436
Education                  341
Marketing                  244
Name: count, dtype: int64

Category in this `df` also has 8 values that happened to be corresponding to those in the required schema.

=> Change to those following the values in the required schema.

In [30]:
category_dict = {'Information Technology':'IT Jobs','Health': 'Healthcare & Nursing Jobs', 'Engineering': 'Engineering Jobs', 
                'Finance': 'Accounting & Finance Jobs', 'Sales': 'Sales Jobs', 'Hospitality': 'Hospitality & Catering',
                 'Education': 'Teaching Jobs', 'Marketing': 'PR, Advertising & Marketing Jobs'}

df['Category'].replace(category_dict,inplace=True)

# examine the fixed Category
df['Category'].value_counts()

Category
IT Jobs                             1251
Healthcare & Nursing Jobs            804
Engineering Jobs                     732
Accounting & Finance Jobs            675
Sales Jobs                           517
Hospitality & Catering               436
Teaching Jobs                        341
PR, Advertising & Marketing Jobs     244
Name: count, dtype: int64

#### 2.6. Salary
`Salary` is calculated by multiplying the `Monthly Payment`

In [32]:
# create Salary column
df['Salary'] = df['Monthly Payment']*12

#### Conflict 7: Open Date and Close Date
Change to datetime format

In [34]:
df['OpenDate'] = pd.to_datetime(df['OpenDate'], utc=True).dt.tz_convert(None)
df['CloseDate'] = pd.to_datetime(df['CloseDate'], utc=True).dt.tz_convert(None)

# examine the data type of OpenDate and CloseDate
df[['OpenDate', 'CloseDate']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   OpenDate   5000 non-null   datetime64[ns]
 1   CloseDate  5000 non-null   datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 78.2 KB


#### Conflict 8: Source Name
Fill the `SourceName` with 'www.jobhuntlisting.com'

In [36]:
# create and fill the sourcename column
df['SourceName'] = 'www.jobhuntlisting.com'

#### Conflict 9: Valid columns and their orders

In [38]:
# examine current df
df.head(3)

,Location,Title,Monthly Payment,CloseDate,Category,ContractTime,OpenDate,Company,Full-Time Equivalent (FTE),Id,ContractType,Salary,SourceName
0,Berkshire,Lead CRA UK,4583.33,2012-03-08 12:00:00,Healthcare & Nursing Jobs,non-specified,2012-01-08 12:00:00,SEC Recruitment,1.0,91469173,full_time,54999.96,www.jobhuntlisting.com
1,Bristol,Possession Manager,2812.50,2013-09-06 12:00:00,Engineering Jobs,permanent,2013-08-07 12:00:00,Navartis Limited,1.0,21088325,full_time,33750.00,www.jobhuntlisting.com
2,Coventry,NVQ Assessor Banking/Financial Services Salary...,1791.67,2013-05-02 00:00:00,Hospitality & Catering,permanent,2013-02-01 00:00:00,Pertemps,1.0,41064446,full_time,21500.04,www.jobhuntlisting.com


`Monthly Payment` and `Full-Time Equivalent (FTE)` are not in the required schema.

In [40]:
# drop unnecessary column
df = df.drop(columns = ['Monthly Payment', 'Full-Time Equivalent (FTE)'])

##### Rearrange the columns order following the required schema

In [42]:
# rearrange the columns
df = df[['Id', 'Title', 'Location', 'Company', 'ContractType', 'ContractTime', 'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName']]

# examine the newly form df
df.head(3)

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,91469173,Lead CRA UK,Berkshire,SEC Recruitment,full_time,non-specified,Healthcare & Nursing Jobs,54999.96,2012-01-08 12:00:00,2012-03-08 12:00:00,www.jobhuntlisting.com
1,21088325,Possession Manager,Bristol,Navartis Limited,full_time,permanent,Engineering Jobs,33750.00,2013-08-07 12:00:00,2013-09-06 12:00:00,www.jobhuntlisting.com
2,41064446,NVQ Assessor Banking/Financial Services Salary...,Coventry,Pertemps,full_time,permanent,Hospitality & Catering,21500.04,2013-02-01 00:00:00,2013-05-02 00:00:00,www.jobhuntlisting.com


### 3. Merging data
Merge two dataframe vertically with `concat`

In [44]:
# concat the df_dataset1 with df
merged_df = pd.concat([df_dataset1, df], ignore_index=True)

In [45]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55753 entries, 0 to 55752
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            55753 non-null  int64  
 1   Title         55753 non-null  object 
 2   Location      55753 non-null  object 
 3   Company       53413 non-null  object 
 4   ContractType  55753 non-null  object 
 5   ContractTime  55753 non-null  object 
 6   Category      55753 non-null  object 
 7   Salary        55753 non-null  float64
 8   OpenDate      55753 non-null  object 
 9   CloseDate     55753 non-null  object 
 10  SourceName    55753 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 4.7+ MB


### 4. Resolving data conflicts:

#### 4.1. Duplicates Check


In [48]:
duplicates = merged_df[merged_df.duplicated()]

print(duplicates)

Empty DataFrame
Columns: [Id, Title, Location, Company, ContractType, ContractTime, Category, Salary, OpenDate, CloseDate, SourceName]
Index: []


There are **no entries** that share the exact data together.

`Id` can be used to be the **global key** due to its uniqueness

=> It needs to be check whether there are duplication in this column
##### Check duplicate Id

In [50]:
duplicates_Id = merged_df[merged_df.duplicated(subset=['Id'])]

print("Duplicates based on Id:")
display(duplicates_Id)

Duplicates based on Id:


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
53192,68996751,Pastry Chef de Partie Food Focused MultiOutle...,Cheshire,Chef Results,full_time,non-specified,Hospitality & Catering,17000.04,2012-01-21 15:00:00,2012-03-21 15:00:00,www.jobhuntlisting.com
53679,67903611,Senior Developer (x ****) Flash/OOD/AS****,Newcastle Upon Tyne,Computappoint,full_time,permanent,IT Jobs,37500.00,2012-06-01 12:00:00,2012-07-01 12:00:00,www.jobhuntlisting.com


##### Handle duplicate Id
- Define a unique 8 digits Id generator that check with the current Ids
- Replace those entries that have duplicate Id with the newly created Id.

In [52]:
def generate_unique_8_digit_id(existing_ids):
    while True:
        new_id = np.random.randint(10000000, 100000000)
        if new_id not in existing_ids:
            return new_id

# replace duplicate IDs with new 8-digit unique IDs
existing_ids = set(merged_df['Id']) # get the set of current Id

for idx in duplicates_Id.index:
    new_id = generate_unique_8_digit_id(existing_ids)
    merged_df.at[idx, 'Id'] = new_id
    existing_ids.add(new_id)  # update the set of existing IDs with the new ID

In [53]:
# check whether there are duplicate Id
duplicates_Id = merged_df[merged_df.duplicated(subset=['Id'])]

print("Number of duplicates based on Id:")
display(duplicates_Id.shape[0])

Number of duplicates based on Id:


0

No entries have duplicate Ids left. 

=> `Id` could be use as global key as the column values are unique.

#### Conflict 2: Check duplication based on the title and company
Jobs advertisement that share the same title and company are consider to be duplicated.

In [56]:
duplicates_jobs = merged_df[merged_df.duplicated(subset=['Title', 'Company'])]
print("Duplicates based on title and company:")
display(duplicates_jobs)

Duplicates based on title and company:


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
37676,64796965,Year 1 Teacher Stanmore,Harrow,ITN Mark Education London City,non-specified,permanent,Teaching Jobs,22000.00,2013-02-13 12:00:00,2013-03-15 12:00:00,tntjobs.co.uk
38775,64796947,Casino VIP Manager,Liverpool,Adam Recruitment,full_time,permanent,"PR, Advertising & Marketing Jobs",40000.00,2012-10-10 15:00:00,2013-01-08 15:00:00,Brand Republic Jobs
50753,91469173,Lead CRA UK,Berkshire,SEC Recruitment,full_time,non-specified,Healthcare & Nursing Jobs,54999.96,2012-01-08 12:00:00,2012-03-08 12:00:00,www.jobhuntlisting.com
50775,44156135,CE I Designer,Warrington,BPS,full_time,permanent,Engineering Jobs,29000.04,2012-12-13 00:00:00,2013-03-13 00:00:00,www.jobhuntlisting.com
50782,39964967,Legal Publishing Account Manager,The City,Euro London Appointments,full_time,permanent,Sales Jobs,25500.00,2012-09-08 12:00:00,2012-10-08 12:00:00,www.jobhuntlisting.com
50824,39781314,Internal Sales Account Manager Wigan ****kOTEBEN,Wigan,Penguin Recruitment,full_time,permanent,Sales Jobs,18500.04,2012-11-29 15:00:00,2012-12-29 15:00:00,www.jobhuntlisting.com
50832,61489480,Renal Healthcare Technician Oxford,Oxford,NaN,full_time,non-specified,Healthcare & Nursing Jobs,26688.00,2013-04-08 12:00:00,2013-06-07 12:00:00,www.jobhuntlisting.com
50850,43213850,IT SALES – ACCOUNT MANAGER ERP & FINANCIAL / ...,West Midlands,Howard Jackson,full_time,permanent,Sales Jobs,57500.04,2013-05-02 12:00:00,2013-07-31 12:00:00,www.jobhuntlisting.com
50853,14547755,Field Sales Rep,Milton Keynes,Gi Group,full_time,permanent,Sales Jobs,22500.00,2013-12-29 12:00:00,2014-03-29 12:00:00,www.jobhuntlisting.com
50861,69129114,Sponsorship Account Manager,London,MAJOR PLAYERS,full_time,contract,Hospitality & Catering,39500.04,2012-01-22 00:00:00,2012-04-21 00:00:00,www.jobhuntlisting.com


These job advertisments are reasonably **dropped**.

In [58]:
merged_df = merged_df.drop_duplicates(subset=['Title', 'Company'])

In [59]:
# check whether there are duplicate job advertisements left
duplicates_jobs = merged_df[merged_df.duplicated(subset=['Title', 'Company'])]
print("Number of duplicates based on title and company:")
display(duplicates_jobs.shape[0])

Number of duplicates based on title and company:


0

### 5. Saving the integrated and reshaped data
Export output data to csv format, named as:
- 'dataset_integrated.csv'

In [61]:
merged_df.to_csv('dataset_integrated.csv', index=False)  